In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## 데이터 확인 및 결측치 확인

In [57]:
tr = pd.read_csv('/kaggle/input/dacon-ddareung/train.csv')
te = pd.read_csv('/kaggle/input/dacon-ddareung/test.csv')
ss = pd.read_csv('/kaggle/input/dacon-ddareung/sample_submission.csv')

### tr데이터 정보 확인

In [58]:
tr.info()

### 데이터 상세 설명
- date_time : 일별 날짜
- wind_direction: 풍향 (degree)
- sky_condition : 하늘 상태 (하단 설명 참조)
- precipitation_form : 강수 형태 (하단 설명 참조)
- wind_speed : 풍속 (m/s)
- humidity : 습도 (%)
- low_temp : 최저 기온 ( `C)
- high_temp : 최고 기온 ( `C)
- Precipitation_Probability : 강수 확률 (%)
- number_of_rentals : 따릉이 대여량
- 
- 기상 데이터는 하루에 총 8번 3시간 간격으로 발표되는 기상단기예보(SHRT) 데이터를 1일 평균으로 변환한 데이터입니다.
- sky_condition (하늘 상태) 코드 : 맑음(1), 구름많음(3), 흐림(4)
- precipitation_form (강수 형태) 코드 : 없음(0), 비(1), 진눈깨비(2), 눈(3), 소나기(4)
* - 위 데이터는 4월~6월의 기상 데이터만을 추출한 것이기 때문에 없음(0), 비(1)만 등장했습니다.
- 따라서 precipitation_form 이 0.5인 경우: "하루의 절반은 비가 올 것으로 예측하고, 나머지 절반은 맑을 것으로 예측했다"는 의미로 해석해주시기 바랍니다.

### 결측치 확인

In [59]:
print('tr 데이터:\n',np.sum(tr.isna()))
print('\n\nte 데이터: \n',np.sum(te.isna()))

## 컬럼 형식 변경 및 변수 생성

###  date_time 컬럼을 년, 월, 일, 요일 컬럼으로 분해

In [60]:
tr.info()

In [61]:
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point))
        month.append(int(month_point))
        day.append(int(day_point))
    dataframe['year'] = year
    dataframe['month'] = month
    dataframe['day'] = day
    dataframe['day_total'] = dataframe['day']
    dataframe['day_total'][dataframe['month'] == 5] += 30
    dataframe['day_total'][dataframe['month'] == 6] += 61
    dataframe['week_day'] = pd.to_datetime(dataframe['date_time']).dt.day_of_week
    week_day_dummies = pd.get_dummies(dataframe['week_day'])
    dataframe = pd.concat([dataframe, week_day_dummies],axis=1)

    ##월0 ~ 일6
    #dataframe["is_weekend"] = dataframe["week_day"].apply(lambda x : 0 if x in (5,6) else 1)
    #pd.to_datetime(dataframe['date_time']).dt.day_name()
    
    # 월은 4,5,6만 있기때문에 더미화
    month_dummies = pd.get_dummies(dataframe['month'],prefix='month')
    dataframe = pd.concat([dataframe, month_dummies],axis=1)
    # 토,일 = 0 , 평일은 1
    del dataframe['date_time']
    #del dataframe['month']
    #del dataframe['week_day']
    return dataframe

tr = seperate_datetime(tr)
te = seperate_datetime(te)

### tr 데이터의 통계치 확인

In [62]:
data_description = tr.describe()
data_description

### 컬럼들간의 통계값 비교

In [63]:
interest_coloumns = ['wind_direction', 'sky_condition', 'precipitation_form', 'wind_speed',
                     'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability',
                     'number_of_rentals']

plt.style.use('fivethirtyeight')
fig, ax =plt.subplots(3, 3, figsize = (20, 12))
fig.suptitle('Histogram of interesting features', fontsize=30)

column_idx = 0
for i in range(3):
    for j in range(3):
        ax[i][j].hist(tr[interest_coloumns[column_idx]], bins=30, color='#eaa18a', edgecolor='#7bcabf')
        ax[i][j].set_title(interest_coloumns[column_idx])
        ax[i][j].axvline(data_description[interest_coloumns[column_idx]]['mean'], c='#f55354', label = f"mean = {round(data_description[interest_coloumns[column_idx]]['mean'], 2)}")
        ax[i][j].axvline(data_description[interest_coloumns[column_idx]]['50%'], c='#518d7d', label = f"median = {round(data_description[interest_coloumns[column_idx]]['50%'], 2)}")
        ax[i][j].legend()
        column_idx += 1

### 년, 월, 일, 주말-평일 별로 대여량 비교

In [68]:
year_rentals = tr.groupby('year').sum()['number_of_rentals']
month_rentals = tr.groupby('month').sum()['number_of_rentals']
day_rentals = tr.groupby('day').sum()['number_of_rentals']
day_total_rentals = tr.groupby('day_total').sum()['number_of_rentals']

In [69]:
fig, ax = plt.subplots(1, 4, figsize=(20, 10))
fig.suptitle('Number of rentals with time scale', fontsize=30)

ax[0].bar(x=['2018', '2019', '2020'], height=year_rentals, color='red', edgecolor='black')
ax[0].set_title('year', fontsize = 20)

ax[1].bar(x = ['4', '5', '6'], height=month_rentals, color='blue', edgecolor='black')
ax[1].set_title('month', fontsize = 20)

ax[2].bar(x=day_rentals.index.to_list(), height= day_rentals, color='orange', edgecolor='black')
ax[2].set_title('days', fontsize = 20)

ax[3].bar(x=day_total_rentals.index.to_list(), height= day_total_rentals, color='black', edgecolor='black')
ax[3].set_title('day_total', fontsize = 20)

plt.show()

### 요일별로 대여량 비교


In [70]:
day_table = tr.groupby('week_day').sum()
#day_table = day_table.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

plt.figure(figsize=(15, 8))
color_map = plt.get_cmap('twilight')
color_list = [color_map.colors[i] for i in range(0, 512, 72)]
plt.bar(day_table.index, day_table.number_of_rentals, edgecolor='black', color=color_list)
plt.xticks(rotation=45, fontsize=15)
plt.title('Total rentals per weekday', fontsize=35)
plt.xlabel('weekday', fontsize=20)
plt.ylabel('number of rentals', fontsize=20)
plt.tight_layout()
plt.show()

### 요일별 대여량은 큰 차이가 없었다

### 년도별 비교

In [71]:
years = [2018, 2019, 2020]
colors = ['#f68842', '#8cc152', '#4a89df']

plt.figure(figsize=(20, 10))

for year, color in zip(years, colors):
    
    precipitation_prb = tr[tr['year'] == year].sort_values(by='Precipitation_Probability')['Precipitation_Probability']
    rentals = tr[tr['year'] == year].sort_values(by='Precipitation_Probability')['number_of_rentals']
    sky_condition = tr[tr['year'] == year].sort_values(by='Precipitation_Probability')['sky_condition']
    
    plt.scatter(precipitation_prb, sky_condition, s=rentals/100, alpha=0.4, c=color, label=f'year={year}')


plt.scatter(precipitation_prb, sky_condition,s=rentals/100, alpha=0.2)
plt.title('Scatter plot Precipitation_Probability & number of rentals', fontsize=30)
plt.xlabel('Precipitation_Probability')
plt.ylabel('sky_condition')
plt.text(20, 1.0, 'The bigger circle size means the more number of rentals', fontsize= 30)
plt.legend()
plt.show()


# 새로운 변수 추가 및 변경

### year 데이터를 0(2018),1(2019),2(2020), 3(2021) 값으로 변경

In [72]:
tr["year"] = tr["year"].apply(lambda x: 0 if x==2018 else (1 if x==2019 else 2))
te['year'] = 3
tr.groupby('year').number_of_rentals.hist()

In [73]:
#새로운 변수 추가

# 기온 = 최저+최고 평균
tr['Temperature'] = (tr['low_temp'] + tr['high_temp'])/2
te['Temperature'] = (te['low_temp'] + te['high_temp'])/2

# 일교차 = 최고 - 최저 기온
tr['temperature_range'] = tr['high_temp'] - tr['low_temp']
te['temperature_range'] = te['high_temp'] - te['low_temp']

# 불쾌지수
tr['Feeling'] = (9/5)*tr['Temperature']-0.55*((100-tr['humidity'])/100)*((9/5)*tr['Temperature']-26)+32
te['Feeling'] = (9/5)*te['Temperature']-0.55*((100-te['humidity'])/100)*((9/5)*te['Temperature']-26)+32

## 데이터 간 상관관계 확인

In [74]:
for i in range(len(tr.columns)):
    print("{} {: .4f}".format(tr.columns[i], tr[tr.columns[i]].corr(tr['number_of_rentals'])))

- 대여량과의 상관관계를 확인해 본 결과,
- year 변수가 가장 높은 상관관계를 보였고 (0.74)
- 그 다음 high_temp(0.42), Temperature (0.38), Feeling(0.37) 등이 있었다

In [75]:
tr_number = tr.select_dtypes(np.number)
tr_number

corr = tr_number.corr()
plt.figure(figsize=(30, 15))
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)

plt.title('Correlation heatmap', fontsize=30)
plt.show()

## 변수 삭제

In [78]:
del tr['month']
del tr['week_day']
del te['month']
del te['week_day']

In [80]:
tr.columns

# 모델링

In [81]:
# metric 정의

import numpy as np

def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

In [88]:
X = tr.drop(['number_of_rentals'], axis=1)#.drop(['wind_direction','Precipitation_Probability','day'], axis=1)
# 풍향과,Precipitation_Probability, day 변수 제거

y = tr['number_of_rentals']

In [89]:
X_combination = X.copy()


# 2. "덥고 습한 날씨"를 알기 위한 정보
X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed'] 

col_list = X_combination.columns

# 이중 for문을 사용하여 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        #if i!=j:
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_combination

## 모델 성능 확인

In [90]:
model = LinearRegression() # 모델 정의
model.fit(X_combination, y) # 학습

y_hat = model.predict(X_combination) # y 예측

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

In [91]:
test = te.copy()

In [92]:
test_X = test

test_X = test_X#.drop(['wind_direction','Precipitation_Probability','day'], axis=1)

# 1. "덥고 습한 날씨"를 알기 위한 정보
test_X['sweat_info'] = test_X['high_temp'] * test_X['humidity'] 

# 2. "춥고 바람부는 날씨"를 알기 위한 정보
test_X['cold_info'] = test_X['low_temp'] * test_X['wind_speed'] 

test_X

In [93]:
# 두 변수간의 곱 변수를 추가합니다.
col_list = test_X.columns

for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        test_X[f'{col_list[i]}*{col_list[j]}'] = test_X[col_list[i]] * test_X[col_list[j]]

In [96]:
len(test_X.columns) == len(X_combination.columns)

In [97]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = model.predict(test_X)

# submission dataFrame 완성
ss['number_of_rentals'] = test_yhat

# 제출 파일 생성
#ss.to_csv('submission.csv', index= False)
ss.to_csv('/kaggle/working/te_submission_11_12_4.csv', index= False)
ss